In [1]:
import os

os.chdir("../..")
import utils

In [2]:
outputfolder = os.getcwd() + '/output/'
datafolder = os.getcwd() + '/datasets/PTB-XL/'
sampling_rate = 100
task = 'subdiagnostic'
experiment_name = 'exp1.1'

In [3]:
data, raw_labels = utils.load_dataset(datafolder, sampling_rate=sampling_rate)
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)
data, labels, Y, _ = utils.select_data(data, labels, task, 0, outputfolder+experiment_name+'/data/')

In [4]:
X_test = data[labels.strat_fold == 10]
y_test = Y[labels.strat_fold == 10]

X_val = data[labels.strat_fold == 9]
y_val = Y[labels.strat_fold == 9]

X_train = data[labels.strat_fold <= 8]
y_train = Y[labels.strat_fold <= 8]

n_classes = y_train.shape[1]

print(f"This experiment has {n_classes} classes")

This experiment has 23 classes


In [5]:
X_train_lead1 = X_train[:,:,0]
X_test_lead1 = X_test[:,:,0]
X_val_lead1 = X_val[:,:,0]

## XGBClassifier

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,MaxPool1D, Convolution1D, Input, BatchNormalization, Flatten
from keras.models import Sequential, Model
from keras.utils import plot_model
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import CSVLogger
import pickle

In [7]:
import tensorflow as tf


def batch_norm():
    return tf.keras.layers.BatchNormalization(momentum=0.9, epsilon=1e-5)

def relu():
    return tf.keras.layers.ReLU()

def dropout(dp=0):
    return tf.keras.layers.Dropout(dp)

def conv1d(filters, kernel_size=3, strides=1):
    return tf.keras.layers.Conv1D(
        filters, kernel_size, strides=strides, padding='same', use_bias=False,
        kernel_initializer=tf.keras.initializers.VarianceScaling())


class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size=3, strides=1, dropout = 0, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = strides
        self.dropout = dropout

    def build(self, input_shape):
        num_chan = input_shape[-1]
        self.conv1 = conv1d(self.filters, self.kernel_size, self.strides)
        self.bn1 = batch_norm()
        self.relu1 = relu()
        self.dropout1 = dropout(self.dropout)
        self.conv2 = conv1d(self.filters, self.kernel_size, 1)
        self.bn2 = batch_norm()
        self.relu2 = relu()
        #self.dropout2 = dropout(self.dropout)
        if num_chan != self.filters or self.strides > 1:
            self.proj_conv = conv1d(self.filters, 1, self.strides)
            self.proj_bn = batch_norm()
            self.projection = True
        else:
            self.projection = False
        super().build(input_shape)

    def call(self, x, **kwargs):
        shortcut = x
        if self.projection:
            shortcut = self.proj_conv(shortcut)
            shortcut = self.proj_bn(shortcut)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x + shortcut)
        #x = self.dropout2(x + shortcut)
        return x


class BottleneckBlock(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size=3, strides=1, expansion=4, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = strides
        self.expansion = expansion

    def build(self, input_shape):
        num_chan = input_shape[-1]
        self.conv1 = conv1d(self.filters, 1, 1)
        self.bn1 = batch_norm()
        self.relu1 = relu()
        self.conv2 = conv1d(self.filters, self.kernel_size, self.strides)
        self.bn2 = batch_norm()
        self.relu2 = relu()
        self.conv3 = conv1d(self.filters * self.expansion, 1, 1)
        self.bn3 = batch_norm()
        self.relu3 = relu()
        if num_chan != self.filters * self.expansion or self.strides > 1:
            self.proj_conv = conv1d(self.filters * self.expansion, 1, self.strides)
            self.proj_bn = batch_norm()
            self.projection = True
        else:
            self.projection = False
        super().build(input_shape)

    def call(self, x, **kwargs):
        shortcut = x
        if self.projection:
            shortcut = self.proj_conv(shortcut)
            shortcut = self.proj_bn(shortcut)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x + shortcut)
        return x


class ResNet(tf.keras.layers.Layer):
    def __init__(self, blocks=(2, 2, 2, 2),
                 filters=(64, 128, 256, 512), kernel_size=(3, 3, 3, 3),
                 block_fn=ResidualBlock, dropout = 0.1, **kwargs):
        super(ResNet, self).__init__(**kwargs)
        self.filters = filters
        self.block_fn=block_fn
        self.block_nums = blocks
        self.kernel_size = kernel_size
        self.dropout = dropout
        self.loss = 'categorical_crossentropy'
        self.model_name = "resnet"

    def build(self, input_shape):
        self.conv1 = conv1d(64, 7, 2)
        self.bn1 = batch_norm()
        self.relu1 = relu()
        self.maxpool1 = tf.keras.layers.MaxPooling1D(3, 2, padding='same')
        self.blocks = []
        for stage, num_blocks in enumerate(self.block_nums):
            for block in range(num_blocks):
                strides = 2 if block == 0 and stage > 0 else 1
                res_block = self.block_fn(self.filters[stage], self.kernel_size[stage], strides, self.dropout)
                self.blocks.append(res_block)
        self.global_pool = tf.keras.layers.GlobalAveragePooling1D()

        super().build(input_shape)

    def get_optimizer(self, lr):
        return tf.keras.optimizers.Adam(lr=lr)

    def call(self, x, **kwargs):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        for res_block in self.blocks:
            x = res_block(x)
        x = self.global_pool(x)
        return x
            
    @staticmethod
    def get_name():
        return 'resnet'

In [8]:
import tensorflow as tf
import os
import numpy as np

class Classifier(tf.keras.Model):
    def __init__(self, model, input_size, n_classes, learning_rate=0.0001, epochs=20, path="temp"):
        super(Classifier, self).__init__()
        self.num_classes = n_classes
        self.learning_rate=learning_rate
        self.input_size = input_size
        self.model = model#tf.keras.Model(inputs=inputs, outputs=model.call(inputs))
        out_act = 'sigmoid' if n_classes == 1 else 'softmax' # change this for multi-label
        self.classifier = tf.keras.layers.Dense(n_classes if n_classes>2 else 1, out_act)
        #os.environ["CUDA_VISIBLE_DEVICES"]="1" # second gpu
        self.epochs = epochs
        self.path = path
        

    def add_compile(self):
        self.compile(optimizer=self.model.get_optimizer(self.learning_rate),
        loss=self.model.loss,
        metrics=['acc', tf.keras.metrics.AUC(multi_label=True)])

    def summary(self):
        input_layer = tf.keras.layers.Input(shape=(self.input_size,1,), dtype='float32')
        model = tf.keras.Model(inputs=input_layer, outputs=self.call(input_layer))
        return model.summary()


    def call(self, x, **kwargs):
        print(x.shape)
        x = self.model(x)
        x = self.classifier(x)
        return x
    
    def fit(self, x, y, validation_data):

        es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

        # x, y = self.transform.process(X=x,labels=y)

        X_val, y_val = validation_data[0], validation_data[1]
        #log_f1 = F1Metric(train=(x, y), validation=(X_val, y_val), path=self.path+os.sep+"models")
        #log_auc = AUCMetric(train=(x, y), validation=(X_val, y_val), path=self.path+os.sep+"models")

        super(Classifier, self).fit(x, y, validation_data = (X_val, y_val), callbacks = [es], epochs = self.epochs, batch_size=128)
        return self

In [14]:
model = ResNet()
model.build(input_shape=1000)
model = Classifier(model=model, input_size=1000, n_classes=23)
model.add_compile()

In [16]:
model.summary()

(None, 1000, 1)
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1000, 1)]         0         
                                                                 
 res_net_2 (ResNet)          (None, 512)               3853504   
                                                                 
 dense_2 (Dense)             (None, 23)                11799     
                                                                 
Total params: 3,865,303
Trainable params: 3,855,703
Non-trainable params: 9,600
_________________________________________________________________


In [15]:
model.fit(X_train_lead1, y_train, validation_data=(X_val_lead1, y_val))

Epoch 1/20
(None, 1000)


ValueError: in user code:

    File "c:\Users\agarw\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\agarw\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\agarw\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\agarw\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\agarw\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\agar

In [13]:
model.summary()

(None, 1000, 1)
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1000, 1)]         0         
                                                                 
 res_net_1 (ResNet)          (None, 512)               3853504   
                                                                 
 dense_1 (Dense)             (None, 23)                11799     
                                                                 
Total params: 3,865,303
Trainable params: 3,855,703
Non-trainable params: 9,600
_________________________________________________________________
